# COGS 189 Final Project

This project aims to </br>
Group Members: 
Stephen Gelinas (A15816513)
Aditya Tomar (A17162996)
Shay Samat
Rolando Restua
Kevin Wong 

## Data Loading

We will first load and inspect the raw EEG data we collected with OpenBCI

In [ ]:
# required imoprts
import pandas as pd
from IPython.display import Image

In [ ]:
# read EEG data
df = pd.read_csv('data/eeg.txt')
df.head()

There appears to be no missing values from the data collection process in the raw EEG data

In [ ]:
# determine if any values are missing
df.isna().sum()

The image below illustrates locations of the 8 selected channels for the data collection process (including GND and REF) 

In [ ]:
Image("data/channels.png", width=400)

## Data Cleaning/Preprocessing

In [ ]:
# "Other" channels that didn't collect EEG data
df[[' Other', ' Other.7']].value_counts().to_frame(name='Total Count')

In [ ]:
# "Analog" channels that didn't collect EEG data
analog = [' Analog Channel 0', ' Analog Channel 1', ' Analog Channel 2']
df[analog].value_counts().to_frame(name='Total Count')

In [ ]:
# drop data from these channels
dropped = [' Other', ' Other.7', ' Analog Channel 0', ' Analog Channel 1', ' Analog Channel 2']
df_cleaned = df.drop(columns=dropped)

In [ ]:
import pandas as pd
import numpy as np
import mne

# Load the data as a pandas DataFrame
df = pd.read_csv('data/eeg.txt')

# Get the sampling frequency from the timestamps
sfreq = 1 / np.mean(np.diff(df[' Timestamp']))

# Convert the data to MNE format
ch_names = df.columns[1:9].tolist()
ch_types = ['eeg'] * len(ch_names)
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
data = df[ch_names].values.T.astype(np.float32)
raw = mne.io.RawArray(data, info)

In [ ]:
raw.plot()

In [ ]:
# Plot raw PSD
raw.plot_psd()

In [ ]:
# Get the accelerometer data
accelerometer_data = df[[' Accel Channel 0', ' Accel Channel 1', ' Accel Channel 2']].values.T.astype(np.float32)

# Calculate the norm of the accelerometer data to get the overall acceleration
acceleration = np.linalg.norm(accelerometer_data, axis=0)

# Identify and remove segments with high acceleration (i.e. head movements)
threshold = np.percentile(acceleration, 95)
bad_segments = np.where(acceleration > threshold)[0]
raw.annotations.append(bad_segments, [1] * len(bad_segments), 'bad')

# Interpolate bad segments
raw.interpolate_bads(reset_bads=True)

# Apply high-pass filter to remove eye movements and remove slow drifts
raw.filter(l_freq=1.0, h_freq=None)

# Apply low-pass filter to remove eye blinks and high-frequency noise

raw.filter(l_freq=0, h_freq=40.0)

# Get the preprocessed data as a numpy array
df_cleaned = raw.get_data()

In [ ]:
raw.plot_psd()

In [ ]:
# Set epoch length to 10 seconds
epoch_length = 9.32

# Create epochs from raw data
epochs = mne.make_fixed_length_epochs(raw, duration=epoch_length, preload=True)
epoch_data = epochs.get_data()
epoch_data

In [ ]:
# Optional: plot epochs to visualize the data
epochs.plot(n_epochs=5)

In [ ]:
# plot the grand average ERP for all channels
erp = epochs.average()
erp.plot()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load preprocessed EEG data from a numpy file
data = df_cleaned

# Select a subset of channels and time points to visualize
channels = [0, 1, 2, 3, 4, 5, 6, 7]
time_points = range(20000, 50000)

# Plot the EEG data
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(data[channels][:, time_points].T)
ax.set_xlabel('Time')
ax.set_ylabel('Amplitude')
ax.set_title('Preprocessed EEG Data')
ax.legend(['Channel {}'.format(i) for i in channels])
plt.show()

In [ ]:
def extract_amplitude(epochs):
    """Extracts amplitude features from EEG epochs across different frequencies.
    
    Parameters
    ----------
    epochs : mne.Epochs
        Epochs object containing the EEG data.
    
    Returns
    -------
    dict
        Dictionary containing the average amplitude values for delta (0-4 Hz), 
        theta (4-8 Hz), alpha (8-12 Hz), beta (12-30 Hz), and gamma (30-100 Hz) 
        frequencies for each channel.
    """
    # Define frequency bands of interest
    freq_bands = {'Delta': (0, 4),
                  'Theta': (4, 8),
                  'Alpha': (8, 12),
                  'Beta': (12, 30),
                  'Gamma': (30, 100)}
    
    # Initialize dictionary to store amplitude features
    amp_features = {}
    
    # Loop over frequency bands and channels to compute average amplitudes
    for band, (fmin, fmax) in freq_bands.items():
        for ch in epochs.ch_names:
            # Extract epochs and compute average amplitude for current band and channel
            band_epochs = epochs.copy().filter(fmin, fmax, picks=ch)
            avg_amp = np.mean(np.abs(band_epochs.get_data()), axis=(0, 2))
            # Add average amplitude to dictionary
            if ch not in amp_features:
                amp_features[ch] = {}
            amp_features[ch][band] = avg_amp
            amp_features
    
    return amp_features

In [ ]:
extract_amplitude(epochs)

In [ ]:
def extract_time_frequency(epochs):
    """Extract time-frequency features from epochs data."""
    # Define frequency bands of interest
    freq_bands = {'theta': [4, 8],
                  'alpha': [8, 12],
                  'beta': [12, 30],
                  'gamma': [30, 80]}

    # Extract time-frequency data using Morlet wavelets
    power = mne.time_frequency.tfr_morlet(epochs, n_cycles=2, freqs=np.arange(2, 81, 2),
                                          use_fft=True, return_itc=False, decim=2, n_jobs=1)
    
    # Compute the average power in each frequency band
    tf_features = dict()
    for band, freq_limits in freq_bands.items():
        freq_mask = np.logical_and(power.freqs >= freq_limits[0], power.freqs < freq_limits[1])
        band_power = power.data[:, freq_mask, :].mean(axis=1)
        tf_features[band] = band_power

    return tf_features


In [ ]:
extract_time_frequency(epochs)

waiting for event id

In [ ]:
def extract_ersp(epochs, event_id):
    """Extracts event-related spectral perturbation (ERSP) from EEG epochs for a specific event type.
    
    Parameters
    ----------
    epochs : mne.Epochs
        Epochs object containing the EEG data.
    event_id : int
        The ID of the event type to calculate ERSP for.
    
    Returns
    -------
    dict
        Dictionary containing the ERSP values for each frequency and time point.
    """
    # Define frequency range and number of cycles for Morlet wavelet
    freqs = np.logspace(*np.log10([3, 35]), num=8)
    n_cycles = freqs / 2.

    # Calculate ERSP using Morlet wavelets
    power, itc = mne.time_frequency.tfr_morlet(epochs[event_id], freqs=freqs, n_cycles=n_cycles,
                                               return_itc=True, average=True, n_jobs=1)
    
    # Get time and frequency information
    times = epochs.times
    frequencies = power.freqs

    # Return ERSP values as dictionary
    return {'times': times, 'frequencies': frequencies, 'power': power.data}


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mne.time_frequency import psd_array_welch

def extract_power_spectrum(epochs):
    """Extracts power spectrum features from EEG epochs across different frequency bands.
    
    Parameters
    ----------
    epochs : mne.Epochs
        Epochs object containing the EEG data.
    
    Returns
    -------
    dict
        Dictionary containing the power spectrum values for delta (0-4 Hz), theta (4-8 Hz), 
        alpha (8-12 Hz), beta (12-30 Hz), and gamma (30-100 Hz).
    """
    freqs = {'delta': (0, 4), 'theta': (4, 8), 'alpha': (8, 12), 'beta': (12, 30), 'gamma': (30, 100)}
    power_spectrum, freq_axis = psd_array_welch(epochs._data, epochs.info['sfreq'], fmin=0, fmax=100, n_fft=2048, 
                                                n_jobs=1, average='median', window='hann', verbose=None)
    power_dict = {}
    for key, value in freqs.items():
        idx = np.where(np.logical_and(freq_axis >= value[0], freq_axis <= value[1]))[0]
        power_dict[key] = np.mean(power_spectrum[:, 7], axis=1)
    
    return power_dict
